In [1]:
import pandas as pd
import datetime

In [14]:
df = pd.read_csv("/Users/florian/Documents/GitHub/4 Semester/APPD/Analyse_von_Prozessdaten/setup/SmA-Four-Tank-Batch-Process_V2.csv", sep= ";")

In [15]:
df["timestamp"]= pd.to_datetime(df["timestamp"], format= "%Y-%m-%d %H:%M:%S")

In [16]:
df_new = df.loc[df["CuStepNo ValueY"] != 2]
df_new["Next_Step"] = df_new["CuStepNo ValueY"].shift()
df_steps = df_new.loc[df_new["CuStepNo ValueY"] != df_new["Next_Step"]]

/var/folders/f6/gs3z1yx9005b5dmpnd_z3t3m0000gn/T/ipykernel_35295/2510048616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Next_Step"] = df_new["CuStepNo ValueY"].shift()


In [17]:
df_steps["time_since_start"] = 0
steps = df_steps.timestamp.unique()
for i in range(len(steps)-1):
    df_steps.loc[df_steps.timestamp == steps[i],"time_since_start"] = (df_new.loc[(df_new.timestamp >= steps[i])&(df_new.timestamp < steps[i+1])].timestamp -steps[i]).max()
#replace all the 0s in time_since_start with the timedelta 0
df_steps.loc[df_steps.time_since_start == 0,"time_since_start"] = datetime.timedelta(0)

/var/folders/f6/gs3z1yx9005b5dmpnd_z3t3m0000gn/T/ipykernel_35295/1473560647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steps["time_since_start"] = 0
/var/folders/f6/gs3z1yx9005b5dmpnd_z3t3m0000gn/T/ipykernel_35295/1473560647.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steps.loc[df_steps.timestamp == steps[i],"time_since_start"] = (df_new.loc[(df_new.timestamp >= steps[i])&(df_new.timestamp < steps[i+1])].timestamp -steps[i]).max()


In [18]:
times = df_steps[df_steps["CuStepNo ValueY"] == 1].timestamp.to_list()
df_test2 = pd.DataFrame(columns=df_new.columns)

In [19]:
for i in range(times.__len__()-1):
    df_test1 = df_new.loc[(df_new.timestamp >=times[i])&(df_new.timestamp <times[i+1])].groupby("CuStepNo ValueY",as_index=False,sort=False).mean()
    df_test1["Batch"] = i
    df_test2 = pd.concat([df_test2,df_test1])

In [20]:
df_test1 = df_new.loc[(df_new.timestamp >=times[261])].groupby("CuStepNo ValueY",as_index=False,sort=False).mean()

In [21]:
df_test2["time_since_start"] = df_steps[:-3].time_since_start.values

In [22]:
df_test2["time_since_start"] = df_test2["time_since_start"].dt.total_seconds()